In [60]:
import torch
from dlc_practical_prologue import generate_pair_sets
import torch.nn as nn

In [69]:
train_input_0, train_target_0, train_classes_0, test_input_0, test_target_0, test_classes_0 = generate_pair_sets(1000)

In [73]:
def normalize_data(tensor):
    mu, std = tensor.mean(), tensor.std()
    tmp = tensor.sub(mu).div(std)
    return tmp

In [71]:
def convert_to_one_hot_labels(target):
    tmp = target.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [97]:
train_input = normalize_data(train_input_0)
# train_target = convert_to_one_hot_labels(train_target_0)
train_target = train_target_0

test_input = normalize_data(test_input_0)
# test_target = convert_to_one_hot_labels(test_target_0)
test_target = test_target_0

In [78]:
print(train_input.mean())
print(train_input.std())

tensor(6.4461e-08)
tensor(1.0000)


In [79]:
print(test_input.mean())
print(test_input.std())

tensor(3.3219e-07)
tensor(1.)


In [80]:
import model

In [81]:
basic_model = model.BaselineNetwork()

In [92]:
def train_model(model, train_input, train_target, train_classes):
    lr, nb_epochs, batch_size = 1e-1, 25, 10 
    
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for e in range(nb_epochs):
        for inputs, targets in zip(train_input.split(batch_size),
                                  train_target.split(batch_size)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [93]:
train_model(basic_model, train_input, train_target, train_classes)

In [94]:
import test

In [ ]:
test.test(train_input, train_target, train_classes, basic_model, nn.CrossEntropyLoss())

In [102]:
test.test(test_input, test_target, test_classes, basic_model, nn.CrossEntropyLoss())

TypeError: can only concatenate str (not "Tensor") to str